# Milestone 4 Checkpoint

LangAlpha is a multi-agent AI equity analysis tool designed to provide comprehensive insights into the stock market. It leverages Large Language Models (LLMs) and agentic workflows to automate data gathering, processing, and analysis.

## Key Technologies

*   **Programming Language:** Python
*   **AI/LLM Frameworks:** LangChain, LangGraph
*   **Core Agent Workflow:** Implemented in `src/agent/market_intelligence_agent` using LangGraph
*   **Data Sources & Tools:**
    *   **Market/Fundamental Data:** Polygon, Yahoo Finance (via `src/mcp_server/market_data.py`, `src/mcp_server/fundamental_data.py`)
    *   **News/Web Research:** Tavily Search, Tickertick News API (via `src/mcp_server/tavily.py`, `src/mcp_server/tickertick.py`)
    *   **Web Browsing:** Playwright (integrated via `browser` agent)
    *   **Code Execution:** Local Python/Bash environment (via `coder` agent)
    *   **Database:** We will use mongoDB to store the query history and the analysis result in production phase


## Market Intelligence Agent Workflow

The primary functionality is delivered through a sophisticated **multi-agent system** built with **LangGraph**, located in `src/agent/market_intelligence_agent/`.

**Workflow Overview:**

1.  **User Query:** The system accepts natural language queries about financial markets, stocks, or economic events.
2.  **Supervisor & Planner:** A `supervisor` agent oversees the entire process. It first consults a `planner` agent, which breaks down the user's request into a detailed, step-by-step research and analysis plan.
3.  **Information Gathering (Parallel & Iterative):**
    *   The `supervisor` delegates tasks based on the plan to specialized agents:
        *   **`researcher`:** Fetches news (Tickertick) and performs general web searches (Tavily) for qualitative information, recent events, and context.
        *   **`market`:** Retrieves quantitative data like stock prices, technical indicators, fundamental data (financials, valuation), and related metrics using dedicated tools connected to Polygon and Yahoo Finance.
        *   **`browser`:** (Used sparingly due to cost) Performs deep web browsing for specific, hard-to-find information on given URLs when the `researcher`'s tools are insufficient.
4.  **Analysis & Synthesis:**
    *   **`coder`:** (If required by the plan) Executes Python code for complex calculations, data manipulation, or analysis beyond the `market` agent's built-in capabilities.
    *   **`analyst`:** Acts as a financial expert (L/S Hedge Fund perspective). It synthesizes the information gathered by all other agents, identifies key insights, assesses risks, generates potential investment theses (Long/Short), and provides actionable financial analysis based *only* on the provided data.
5.  **Reporting:**
    *   The `supervisor` reviews the findings from all agents, potentially iterating or requesting clarifications.
    *   Finally, a `reporter` agent compiles all the validated information and analysis into a comprehensive, well-structured Markdown report, including tables and summaries.

**Key Capabilities:**

*   **Autonomous Research:** Agents collaboratively gather diverse data types (news, market data, fundamentals, web content).
*   **In-depth Analysis:** Combines quantitative data retrieval with qualitative research and expert financial analysis.
*   **Structured Planning:** Ensures a logical flow of information gathering and analysis tailored to the user's query.
*   **Flexible Orchestration:** The `supervisor` dynamically routes tasks and manages the workflow, allowing for iteration and refinement.
*   **Actionable Insights:** Aims to provide not just data, but synthesized analysis suitable for investment decision-making context (though not direct financial advice).

## Damodaran Valuation Model:
- Implements valuation methodologies inspired by Professor Aswath Damodaran.
- Provides a user interface (details TBD) for manual input and valuation generation.
- See the [ginzu_interface.ipynb](https://github.com/Chen-zexi/LangAlpha/blob/master/notebooks/ginzu_interface.ipynb) for details.

## Trading Strategy:
- See the [counter-trend-trading-strategy colab](https://colab.research.google.com/drive/1Wo1f5SvZ3M9YjUx7gl4q2rSIo7PMegBN?usp=sharing#scrollTo=1kA-HcwGkK9Z) for details.



## Important Notes for Graders
- **DO NOT RUN THIS NOTEBOOK**: The source code are not accessible when submitting a single notebook
- The complete codebase can be accessed at [GitHub Repository](https://github.com/Chen-zexi/LangAlpha)

Below is an image demonstrate the current agent workflow
![graph](https://github.com/Chen-zexi/LangAlpha/blob/master/assets/graph.png)

In [ ]:
import sys
import os
import warnings
# Add the src directory to the Python path and suppress warnings
sys.path.append(os.path.abspath('../../'))
warnings.filterwarnings("ignore", category=FutureWarning)
from src.agent.market_intelligence_agent.service.workflow_service_dev import run_workflow

In [2]:
raw_response, final_report = await run_workflow(user_query="What is the market overview for the week of April 22, 2025?")

2025-04-27 10:27:49,789 - httpx - INFO - HTTP Request: POST http://localhost:2024/threads "HTTP/1.1 200 OK"
2025-04-27 10:27:49,795 - httpx - INFO - HTTP Request: POST http://localhost:2024/threads/25575dee-7a78-4390-8288-71d90c23009b/runs/stream "HTTP/1.1 200 OK"


Starting graph stream for query: 'What is the market overview for the week of April 22, 2025?'...
------------------------------------------------------------------------------------------------------------------------
Coordinator handed off the task to the team.
------------------------------------------------------------------------------------------------------------------------
Planner is thinking...
------------------------------------------------------------------------------------------------------------------------
Market Overview for the Week of April 22, 2025
------------------------------------------------------------------------------------------------------------------------
The user is requesting a market overview for the week of April 22, 2025. To provide a comprehensive and valuable response, it is important to analyze major stock indices, key sector performances, significant market-moving events, macroeconomic data releases, and notable corporate news during this perio

## Formate the response with Markdown

In [3]:
from IPython.display import Markdown, display

def display_markdown(content):
    start_marker = "<response>\n"
    end_marker = "</response>\n"
    start_index = content.find(start_marker)
    end_index = content.find(end_marker)
    if start_index != -1 and end_index != -1:
        content = content[start_index + len(start_marker):end_index]
    return Markdown(content)
    
    
# Display the formatted markdown
display(display_markdown(final_report))

# Market Overview Report  
### Week of April 22, 2025

---

## Executive Summary

- **Major U.S. indices posted strong weekly gains:**  
  - *S&P 500 (SPY): +5.86%*  
  - *Dow Jones (DIA): +3.87%*  
  - *Nasdaq (QQQ): +7.69%*  

- **Key sector ETFs outperformed broadly:**  
  - *Technology (XLK): +9.5%* (highest gain)  
  - *Consumer Discretionary (XLY): +8.26%*  
  - *Financials (XLF): +3.63%*  

- **Market drivers included:**  
  - Robust corporate earnings (Equifax, 3M) supporting sector gains  
  - Easing tariff tensions boosting market sentiment  
  - Eurozone consumer confidence fell sharply (-16.7), indicating geopolitical risks  

- **Notable anomaly:** Technology sector surged (+9.5%) despite mixed earnings (Tesla weak, 3M strong), likely driven by optimism on trade developments and growth prospects.  

- **Investment insight:**  
  - Analysts recommend *initiating long positions* in Technology and Consumer Discretionary sectors with medium-to-high conviction.  
  - Risk management is crucial due to geopolitical uncertainties and earnings variability.  

---

## Market Context

The week starting April 22, 2025, was marked by a broad and steady advance in U.S. equity markets amid a combination of strong corporate earnings, signs of easing international trade tensions, and moderated volatility. While positive sentiment predominated, external risks lingered as Eurozone consumer sentiment deteriorated markedly, reflecting rising household pessimism and geopolitical concerns.

---

## Key Findings

| Metric / Sector               | Opening Price (Apr 22) | Closing Price (Apr 25) | Weekly % Change | Volume (M) / Notes                    |
|------------------------------|-----------------------|-----------------------|-----------------|-------------------------------------|
| **SPY (S&P 500 ETF)**         | 520.14                | 550.64                | +5.86%          | Moderate volatility (10-18% annualized) |
| **DIA (Dow Jones ETF)**       | 386.07                | 401.02                | +3.87%          |                                     |
| **QQQ (Nasdaq ETF)**          | 438.82                | 472.56                | +7.69%          | Strongest gains                     |
| **XLF (Financials ETF)**      | N/A                   | N/A                   | +3.63%          | Earnings boost from Equifax         |
| **XLK (Technology ETF)**      | N/A                   | N/A                   | +9.50%          | Mixed earnings; strong volume (4.82M) |
| **XLY (Consumer Discretionary ETF)** | N/A            | N/A                   | +8.26%          | Stable volume (3.60M)               |

---

## Detailed Analysis

### Corporate Earnings Impact

- **Positive contributors:**  
  - *Equifax* delivered strong quarterly results, bolstering the Financials sector.  
  - *3M* reported solid earnings, contributing to Industrials and technology-related optimism.

- **Underperformance:**  
  - *Tesla*’s earnings fell short, creating a contrasting signal within Technology but not deterring the sector's overall robust weekly advance.

### Tariff Tensions and Macro Influence

- Easing of tariff tensions was a critical market sentiment driver, reducing uncertainty affecting multinational corporations and boosting risk appetite across sectors.

- This macro tailwind helped offset negative external factors, maintaining bullish momentum particularly noticeable in the Technology sector, which often is sensitive to trade policies.

### Eurozone Consumer Confidence

- The Eurozone’s consumer confidence dropped sharply to -16.7, indicating elevated household pessimism and geopolitical risks.

- Despite this, U.S. markets showed resilience, indicating that domestic and trade-related factors took precedence over external macro concerns during the week.

### Technology Sector Anomaly

- Technology (XLK) recorded the highest weekly gain (+9.5%) despite uneven earnings reports.

- This suggests that forward-looking optimism, driven by tariff easing and growth expectations, outweighed short-term earnings disappointments, evidencing strong investor appetite for tech growth themes.

---

## Future Outlook

- **Positive catalysts:**  
  - Continued clarity and potential further easing in international trade policies.  
  - Upcoming earnings seasons may validate current optimism or reveal stresses.  
  - U.S. economic data releases could reinforce positive trends.

- **Potential headwinds:**  
  - Geopolitical tensions and macro risks in Europe may escalate, weighing on global risk sentiment.  
  - Earnings misses similar to Tesla’s could broaden, especially in high-growth sectors, prompting caution.  
  - Market volatility may increase if present positive factors lose momentum.

---

## Conclusions and Recommendations

| Aspect                   | Summary / Action                                 |
|--------------------------|------------------------------------------------|
| **Investment Position**  | Initiate *long positions* in Technology (XLK) and Consumer Discretionary (XLY) sectors, capitalizing on earnings strength and tariff optimism. |
| **Entry Guidance**       | Enter near recent volume-supported levels; monitor for minor pullbacks. |
| **Target Gains**         | 10-15% upside in Technology; 8-12% in Consumer Discretionary in medium term (1-3 months). |
| **Risk Controls**        | Use stop-loss orders 3-5% below entry to mitigate downside from geopolitical or earnings risks. |
| **Monitoring Needs**     | Closely track geopolitical developments, tariff policy shifts, Eurozone data, and earnings updates for potential market reversals. |

---

## Summary Table: Weekly Performance Highlights

| Index / Sector         | Weekly % Change | Primary Drivers                                       | Notes                        |
|-----------------------|-----------------|------------------------------------------------------|------------------------------|
| S&P 500 (SPY)          | +5.86%          | Broad earnings strength, tariff optimism             | Steady appreciation          |
| Dow Jones (DIA)        | +3.87%          | Financials earnings (Equifax), stable macro backdrop | Moderate gain                |
| Nasdaq (QQQ)           | +7.69%          | Technology rally despite mixed earnings               | Highest weekly return        |
| Financials (XLF)       | +3.63%          | Strong corporate earnings                             | Volume: 35.51M               |
| Technology (XLK)       | +9.50%          | Tariff easing optimism, mixed earnings               | Volume: 4.82M                |
| Consumer Discretionary (XLY) | +8.26%     | Resilient consumer demand                             | Stable volume                |

---

> *“The week demonstrated strong investor confidence in domestic economic drivers and easing trade frictions, underpinning significant gains across major U.S. stock indices and key sectors despite geopolitical and external macroeconomic headwinds.”*

---

*Report prepared exclusively on market data and information from the week of April 22, 2025.*